In [46]:
!pip uninstall dgl -y

Found existing installation: dgl 2.1.0
Uninstalling dgl-2.1.0:
  Successfully uninstalled dgl-2.1.0


In [1]:
# Your code that installs packages goes here
! pip install torch
#! pip install dgl -f https://data.dgl.ai/wheels/repo.html
#! pip install dgl-cu101==0.6.1
! pip install  dgl -f https://data.dgl.ai/wheels/cu121/repo.html
! pip install pandas
! pip install pyyaml
! pip install pydantic

  Using cached torch-2.3.0-cp311-none-macosx_11_0_arm64.whl.metadata (26 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl.metadata (3.0 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.3.0-cp311-none-macosx_11_0_arm64.whl (61.0 MB)
Using cached filelock-3.13.4-py3-none-any.whl (11 kB)
Using cached fsspec-2024.3.1-py3-none-any.whl (171 kB)
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
Using cached networkx-3.3-py3-none-any.whl (1.7 MB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
Using cached MarkupSafe-2.1.5-cp311-cp311-macosx_10_9_universal2.whl (18 kB)
Using cached mpmath-1.3.0-py3-none-any.whl (5

In [7]:
import torch
print(torch.__version__)

2.3.0


In [8]:
import torch
import dgl.nn as dglnn

ModuleNotFoundError: No module named 'dgl'

In [ ]:
import dgl
import dgl.function as fn
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:

class RGCN(nn.Module):
    def __init__(self, in_feat, hidden_feat, out_feat, num_rels):
        super(RGCN, self).__init__()
        self.W = nn.ModuleList([
            nn.Linear(in_feat, hidden_feat) for _ in range(num_rels*2)
        ])
        self.loop_W = nn.Linear(in_feat, hidden_feat)
        self.out_W = nn.Linear(hidden_feat, out_feat)

    def forward(self, g, feats):
        g = g.local_var()
        g.ndata['h'] = feats
        
        # Apply relation-specific weights
        for i in range(g.number_of_relations()):
            g.apply_edges(fn.copy_u('h', 'msg'), etype=g.etypes[i])
            g.send_and_recv((g.edges(etypes=g.etypes[i], form='uv')[0],
                            g.edges(etypes=g.etypes[i], form='uv')[1]),
                            [fn.mean('msg', 'h')], 
                            [self.W[i]], 
                            g.etypes[i])
        
        # Self-loop
        loop_messages = self.loop_W(g.ndata['h'])
        g.ndata['h'] = g.ndata['h'] + loop_messages
        
        # Output projection
        output = self.out_W(g.ndata['h'])
        
        return output